In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import ascii
from scipy.optimize import curve_fit
from lmfit import Model, minimize, Parameters, report_fit
from lmfit.models import GaussianModel, PolynomialModel
import astropy.units as u
import pandas as pd
import glob
import pandas as pd
import csv
import celerite

In [2]:
def Gauss(x, h1, c1, w1):
    G = h1*np.exp(-(x-c1)**2/(2*w1**2))
    return G

def doubleGauss(x, h1, c1, w1, h2, c2, w2):
    G = h1*np.exp(-(x-c1)**2/(2*w1**2)) + h2*np.exp(-(x-c2)**2/(2*w2**2))
    return G

def tripleGauss(x, h1, c1,w1, h2, c2, w2, h3, c3, w3):
    G = h1*np.exp(-(x-c1)**2/(2*w1**2)) + h2*np.exp(-(x-c2)**2/(2*w2**2)) + h3*np.exp(-(x-c3)**2/(2*w3**2))
    return G


In [3]:
def fit_coord(l, b, n_HI = 1, n_65 = 1, n_67 = 1, n_CO = 1, HI_p0 = [1, 0, 1], OH65_p0 = [0.03, 0, 4], OH67_p0 = [0.03, 0, 4], CO_p0 = [1, 0, 1], HI_bounds = (-np.inf, np.inf), OH65_bounds = (-np.inf, np.inf), OH67_bounds = (-np.inf, np.inf), CO_bounds = (-np.inf, np.inf), Cloud = []):
    
    # Setup a pandas dataframe to save all the variables we'd like
    
    '''Purpose:
    This function fits multiple gaussians to HI, CO and HI data and returns the parameters...
    
    Parameters:
    l: galactic latitude, int
    b: galactic lontitude, int
    HIdata: np.array of HI data in Tb, vel
    OHdata: np.array of OH data in Tb, vel
    ...

    Returns a pandas dataframe
    '''
    OH67 = ascii.read('C:/Users/seven/OneDrive/Documents/DustSurvey/OH' + str(l) + 'p' + str(b) + '_67basesub.txt')
    OH65 = ascii.read('C:/Users/seven/OneDrive/Documents/DustSurvey/OH' + str(l) + 'p' + str(b) + '_65basesub.txt')
    HI = ascii.read('C:/Users/seven/OneDrive/Documents/DustSurvey/HI' + str(l) + 'p' + str(b) + '.txt', data_start = 13, data_end = 167)
    CO = ascii.read('C:/Users/seven/OneDrive/Documents/DustSurvey/CO' + str(l) + 'p' + str(b) + '.txt')

    


    resultsTable = pd.DataFrame(np.array([[0, 
                                         0, 
                                         0,
                                         0,
                                         0,
                                         0,
                                         0,
                                         0, 
                                         0,
                                         0,
                                         0]]),
                               columns=['l (deg)', 
                                        'b (deg)', 
                                        'line',
                                        'comp',
                                        'T_peak (K)',
                                        'T_error (K)', 
                                        'CenterVel (km/s)',
                                        'CenterVel_error (km/s)',
                                        'FWHM (km/s)',
                                        'FWHM error (km/s)',
                                        'Cloud'
                                       ])

    
    velHI = HI['col1']
    TaHI = HI['col2']
    vel67 = OH67['col1']
    Ta67 = OH67['col2']
    vel65 = OH65['col1']
    Ta65 = OH65['col2']
    velCO = CO['col1']
    TaCO = CO['col2']
    sigT67 = np.std(Ta67[np.where(vel67< -50)])
    sigT65 = np.std(Ta65[np.where(vel65< -50)])
    sigTCO = np.std(TaCO[np.where(velCO< -10)])
    
    size67 = len(vel67)
    size65 = len(vel65)
    sizeHI = len(velHI)
    sizeCO = len(velCO)
    
    sig = 0.003
    #print(vel67[1]-vel67[0])

    #fit HI data
    if n_HI == 1:

        parametersHI, covarianceHI = curve_fit(Gauss, velHI, TaHI, HI_p0, bounds = HI_bounds)

        fitHIh = [parametersHI[0]]
        fitHIc = [parametersHI[1]]
        fitHIw = [parametersHI[2]]
        perrHI = np.sqrt(np.diag(covarianceHI))
        errorHIh = [perrHI[0]]
        errorHIc = [perrHI[1]]
        errorHIw = [perrHI[2]]
        
        for i in range(n_HI):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        'HI',
                                                        i+1,
                                                        fitHIh[i],
                                                        errorHIh[i],
                                                        fitHIc[i],
                                                        errorHIc[i],
                                                        fitHIw[i],
                                                        errorHIw[i],
                                                        Cloud
                                                        ]

        
    if n_HI == 2:
        
        HI_p0 = np.tile(HI_p0, n_HI)
        HI_bounds = np.tile(HI_bounds, n_HI)
        parametersHI, covarianceHI = curve_fit(doubleGauss, velHI, TaHI, HI_p0, bounds = HI_bounds)

        fitHIh = [parametersHI[0], parametersHI[3]]
        fitHIc = [parametersHI[1], parametersHI[4]]
        fitHIw = [parametersHI[2], parametersHI[5]]
        perrHI = np.sqrt(np.diag(covarianceHI))
        errorHIh = [perrHI[0], perrHI[3]]
        errorHIc = [perrHI[1], perrHI[4]]
        errorHIw = [perrHI[2], perrHI[5]]
        
        
        for i in range(n_HI):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        'HI',
                                                        i+1,
                                                        fitHIh[i],
                                                        errorHIh[i],
                                                        fitHIc[i],
                                                        errorHIc[i],
                                                        fitHIw[i],
                                                        errorHIw[i],
                                                        Cloud
                                                        ]
            
    if n_HI == 3:
        
        HI_p0 = np.tile(HI_p0, n_HI)
        HI_bounds = np.tile(HI_bounds, n_HI)
        parametersHI, covarianceHI = curve_fit(tripleGauss, velHI, TaHI, HI_p0, bounds = HI_bounds)
        
        fitHIh = [parametersHI[0], parametersHI[3], parametersHI[6]]
        fitHIc = [parametersHI[1], parametersHI[4], parametersHI[7]]
        fitHIw = [parametersHI[2], parametersHI[5], parametersHI[8]]
        perrHI = np.sqrt(np.diag(covarianceHI))
        errorHIh = [perrHI[0], perrHI[3], perrHI[6]]
        errorHIc = [perrHI[1], perrHI[4], perrHI[7]]
        errorHIw = [perrHI[2], perrHI[5], perrHI[8]]

        for i in range(n_HI):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        'HI',
                                                        i+1,
                                                        fitHIh[i],
                                                        errorHIh[i],
                                                        fitHIc[i],
                                                        errorHIc[i],
                                                        fitHIw[i],
                                                        errorHIw[i],
                                                        Cloud
                                                        ]

    #fit 67 data
    
    if n_67 == 1:
        parameters67, covariance67 = curve_fit(Gauss, vel67, Ta67, OH67_p0, bounds = OH67_bounds, sigma = [sig] * size67, absolute_sigma = True)
        
        fit67h = [parameters67[0]]
        fit67c = [parameters67[1]]
        fit67w = [parameters67[2]]
        perr67 = np.sqrt(np.diag(covariance67))
        error67h = [perr67[0]]
        error67c = [perr67[1]]
        error67w = [perr67[2]]
        
        r = n_HI - n_67
        
        if r == 0:

            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            '67',
                                                            i+1,
                                                            fit67h[i],
                                                            error67h[i],
                                                            fit67c[i],
                                                            error67c[i],
                                                            fit67w[i],
                                                            error67w[i],
                                                            Cloud
                                                            ]
        elif r == 1:
            fit67h = [parameters67[0], sigT67]
            fit67c = [parameters67[1], parametersHI[1]]
            fit67w = [parameters67[2], parametersHI[2]]
            error67h = [perr67[0], 0]
            error67c = [perr67[1], 0]
            error67w = [perr67[2], 0]
            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            '67',
                                                            i+1,
                                                            fit67h[i],
                                                            error67h[i],
                                                            fit67c[i],
                                                            error67c[i],
                                                            fit67w[i],
                                                            error67w[i],
                                                            Cloud
                                                            ]       
        elif r == 2:
            fit67h = [parameters67[0], sigT67, sigT67]
            fit67c = [parameters67[1], parametersHI[1], parametersHI[4]]
            fit67w = [parameters67[2], parametersHI[2], parametersHI[5]]
            error67h = [perr67[0], 0, 0]
            error67c = [perr67[1], 0, 0]
            error67w = [perr67[2], 0, 0]
            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            '67',
                                                            i+1,
                                                            fit67h[i],
                                                            error67h[i],
                                                            fit67c[i],
                                                            error67c[i],
                                                            fit67w[i],
                                                            error67w[i],
                                                            Cloud
                                                            ]     
        else:
            print("Uh oh this ain't workin")
        
    if n_67 == 2:
        
        OH67_p0 = np.tile(OH67_p0, n_67)
        OH67_bounds = np.tile(OH67_bounds, n_67)
        parameters67, covariance67 = curve_fit(doubleGauss, vel67, Ta67, OH67_p0, bounds = OH67_bounds, sigma = [sig] * size67, absolute_sigma = True)

        fit67h = [parameters67[0], parameters67[3]]
        fit67c = [parameters67[1], parameters67[4]]
        fit67w = [parameters67[2], parameters67[5]]
        perr67 = np.sqrt(np.diag(covariance67))
        error67h = [perr67[0], perr67[3]]
        error67c = [perr67[1], perr67[4]]
        error67w = [perr67[2], perr67[5]]
        
        
        r = n_HI - n_67
            

        if r == 0:
            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            '67',
                                                            i+1,
                                                            fit67h[i],
                                                            error67h[i],
                                                            fit67c[i],
                                                            error67c[i],
                                                            fit67w[i],
                                                            error67w[i],
                                                            Cloud
                                                            ]
                
        elif r==1:
            fit67h = [parameters67[0], parameters67[3], sigT67]
            fit67c = [parameters67[1], parameters67[4], parametersHI[1]]
            fit67w = [parameters67[2], parameters67[5], parametersHI[2]]
            error67h = [perr67[0],  perr67[3], 0]
            error67c = [perr67[1],  perr67[4], 0]
            error67w = [perr67[2],  perr67[5], 0]
            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            '67',
                                                            i+1,
                                                            fit67h[i],
                                                            error67h[i],
                                                            fit67c[i],
                                                            error67c[i],
                                                            fit67w[i],
                                                            error67w[i],
                                                            Cloud
                                                            ]

            
    if n_67 == 3:
        
        OH67_p0 = np.tile(OH67_p0, n_67)
        OH67_bounds = np.tile(OH67_bounds, n_67)
        parameters67, covariance67 = curve_fit(tripleGauss, vel67, Ta67, OH67_p0, bounds = OH67_bounds, sigma = [sig] * size65, absolute_sigma = True) 
    
        fit67h = [parameters67[0], parameters67[3], parameters67[6]]
        fit67c = [parameters67[1], parameters67[4], parameters67[7]]
        fit67w = [parameters67[2], parameters67[5], parameters67[8]]
        perr67 = np.sqrt(np.diag(covariance67))
        error67h = [perr67[0], perr67[3], perr67[6]]
        error67c = [perr67[1], perr67[4], perr67[7]]
        error67w = [perr67[2], perr67[5], perr67[8]]

                
        for i in range(n_67):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        '67',
                                                        i+1,
                                                        fit67h[i],
                                                        error67h[i],
                                                        fit67c[i],
                                                        error67c[i],
                                                        fit67w[i],
                                                        error67w[i],
                                                        Cloud
                                                        ]
            
    #fit 65 data
    
    if n_65 == 1:
        parameters65, covariance65 = curve_fit(Gauss, vel65, Ta65, OH65_p0, bounds = OH65_bounds, sigma = [sig] * size65, absolute_sigma = True)
        
        fit65h = [parameters65[0]]
        fit65c = [parameters65[1]]
        fit65w = [parameters65[2]]
        perr65 = np.sqrt(np.diag(covariance65))
        error65h = [perr65[0]]
        error65c = [perr65[1]]
        error65w = [perr65[2]]
        
        s = n_HI - n_65
        
        if s == 0:

            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            '65',
                                                            i+1,
                                                            fit65h[i],
                                                            error65h[i],
                                                            fit65c[i],
                                                            error65c[i],
                                                            fit65w[i],
                                                            error65w[i],
                                                            Cloud
                                                            ]
        elif s == 1:
            fit65h = [parameters65[0], sigT65]
            fit65c = [parameters65[1], parametersHI[1]]
            fit65w = [parameters65[2], parametersHI[2]]
            error65h = [perr65[0], 0]
            error65c = [perr65[1], 0]
            error65w = [perr65[2], 0]
            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            '65',
                                                            i+1,
                                                            fit65h[i],
                                                            error65h[i],
                                                            fit65c[i],
                                                            error65c[i],
                                                            fit65w[i],
                                                            error65w[i],
                                                            Cloud
                                                            ]       
        elif s == 2:
            fit65h = [parameters65[0], sigT65, sigT65]
            fit65c = [parameters65[1], parametersHI[1], parametersHI[4]]
            fit65w = [parameters65[2], parametersHI[2], parametersHI[5]]
            error65h = [perr65[0], 0, 0]
            error65c = [perr65[1], 0, 0]
            error65w = [perr65[2], 0, 0]
            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            '65',
                                                            i+1,
                                                            fit65h[i],
                                                            error65h[i],
                                                            fit65c[i],
                                                            error65c[i],
                                                            fit65w[i],
                                                            error65w[i],
                                                            Cloud
                                                            ]     
    
    if n_65 == 2:
        
        OH65_p0 = np.tile(OH65_p0, n_65)
        OH65_bounds = np.tile(OH65_bounds, n_65)
        parameters65, covariance65 = curve_fit(doubleGauss, vel65, Ta65, OH65_p0, bounds = OH65_bounds, sigma = [sig] * size65, absolute_sigma = True)
    
        fit65h = [parameters65[0], parameters65[3]]
        fit65c = [parameters65[1], parameters65[4]]
        fit65w = [parameters65[2],parameters65[4]]
        perr65 = np.sqrt(np.diag(covariance65))
        print(perr65)
        error65h = [perr65[0], perr65[3]]
        error65c = [perr65[1], perr65[4]]
        error65w = [perr65[2], perr65[5]]
        
        
        s = n_HI - n_65
        
        if s == 0:

            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            '65',
                                                            i+1,
                                                            fit65h[i],
                                                            error65h[i],
                                                            fit65c[i],
                                                            error65c[i],
                                                            fit65w[i],
                                                            error65w[i],
                                                            Cloud
                                                            ]
        elif s == 1:
            fit65h = [parameters65[0], parameters65[3], sigT65]
            fit65c = [parameters65[1], parameters65[4], parametersHI[1]]
            fit65w = [parameters65[2], parameters65[4], parametersHI[2]]
            error65h = [perr65[0], perr65[3], 0]
            error65c = [perr65[1], perr65[4], 0]
            error65w = [perr65[2], perr65[5], 0]
            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            '65',
                                                            i+1,
                                                            fit65h[i],
                                                            error65h[i],
                                                            fit65c[i],
                                                            error65c[i],
                                                            fit65w[i],
                                                            error65w[i],
                                                            Cloud
                                                            ]       
            
    if n_65 == 3:
        
        OH65_p0 = np.tile(OH65_p0, n_65)
        OH65_bounds = np.tile(OH65_bounds, n_65)
        parameters65, covariance65 = curve_fit(tripleGauss, vel65, Ta65, OH65_p0, bounds = OH65_bounds, sigma = [sig] * size65, absolute_sigma = True)    
        
        fit65h = [parameters65[0], parameters65[3], parameters65[6]]
        fit65c = [parameters65[1], parameters65[4], parameters65[7]]
        fit65w = [parameters65[2], parameters65[5], parameters65[8]]
        perr65 = np.sqrt(np.diag(covariance65))
        error65h = [perr65[0], perr65[3], perr65[6]]
        error65c = [perr65[1], perr65[4], perr65[7]]
        error65w = [perr65[2], perr65[5], perr65[8]]
        
        
        for i in range(n_65):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        '65',
                                                        i+1,
                                                        fit65h[i],
                                                        error65h[i],
                                                        fit65c[i],
                                                        error65c[i],
                                                        fit65w[i],
                                                        error65w[i],
                                                        Cloud
                                                        ]




            
    #fit CO data
    
    if n_CO == 1:
        parametersCO, covarianceCO = curve_fit(Gauss, velCO, TaCO, CO_p0,bounds =  CO_bounds)
            
        fitCOh = [parametersCO[0]]
        fitCOc = [parametersCO[1]]
        fitCOw = [parametersCO[2]]
        perrCO = np.sqrt(np.diag(covarianceCO))
        errorCOh = [perrCO[0]]
        errorCOc = [perrCO[1]]
        errorCOw = [perrCO[2]]
               
        q = n_HI - n_CO
        
        if q == 0:

            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            'CO',
                                                            i+1,
                                                            fitCOh[i],
                                                            errorCOh[i],
                                                            fitCOc[i],
                                                            errorCOc[i],
                                                            fitCOw[i],
                                                            errorCOw[i],
                                                            Cloud
                                                            ]
        elif q == 1:
            fitCOh = [parametersCO[0], sigTCO]
            fitCOc = [parametersCO[1], parameters65[1]]
            fitCOw = [parametersCO[2], parameters65[2]]
            errorCOh = [perrCO[0], 0]
            errorCOc = [perrCO[1], 0]
            errorCOw = [perrCO[2], 0]
            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            'CO',
                                                            i+1,
                                                            fitCOh[i],
                                                            errorCOh[i],
                                                            fitCOc[i],
                                                            errorCOc[i],
                                                            fitCOw[i],
                                                            errorCOw[i],
                                                            Cloud
                                                            ]       
        elif q == 2:
            fitCOh = [parametersCO[0], sigTCO, sigTCO]
            fitCOc = [parametersCO[1], parameters65[1], parameters65[1]]
            fitCOw = [parametersCO[2], parameters65[2], parameters65[2]]
            errorCOh = [perrCO[0], 0, 0]
            errorCOc = [perrCO[1], 0, 0]
            errorCOw = [perrCO[2], 0, 0]
            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            'CO',
                                                            i+1,
                                                            fitCOh[i],
                                                            errorCOh[i],
                                                            fitCOc[i],
                                                            errorCOc[i],
                                                            fitCOw[i],
                                                            errorCOw[i],
                                                            Cloud
                                                            ]     

    
        
    if n_CO == 2:
        
        CO_p0 = np.tile(CO_p0, n_CO)
        CO_bounds = np.tile(CO_bounds, n_CO)
        parametersCO, covarianceCO = curve_fit(doubleGauss, velCO, TaCO, CO_p0, bounds = CO_bounds)
    
        fitCOh = [parametersCO[0], parametersCO[3]]
        fitCOc = [parametersCO[1], parametersCO[4]]
        fitCOw = [parametersCO[2], parametersCO[5]] 
        perrCO = np.sqrt(np.diag(covarianceCO))
        errorCOh = [perrCO[0], perrCO[3]]
        errorCOc = [perrCO[1], perrCO[4]]
        errorCOw = [perrCO[2], perrCO[5]]
        
        q = n_HI - n_CO
        
        if q == 0:

            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            'CO',
                                                            i+1,
                                                            fitCOh[i],
                                                            errorCOh[i],
                                                            fitCOc[i],
                                                            errorCOc[i],
                                                            fitCOw[i],
                                                            errorCOw[i],
                                                            Cloud
                                                            ]
        elif q == 1:
            fitCOh = [parametersCO[0], parametersCO[3], sigTCO]
            fitCOc = [parametersCO[1], parametersCO[4], parameters65[1]]
            fitCOw = [parametersCO[2], parametersCO[5], parameters65[2]]
            errorCOh = [perrCO[0], perrCO[3], 0]
            errorCOc = [perrCO[1], perrCO[4], 0]
            errorCOw = [perrCO[2], perrCO[5], 0]
            for i in range(n_HI):
                resultsTable.loc[len(resultsTable.index)] = [ l,
                                                            b,
                                                            'CO',
                                                            i+1,
                                                            fitCOh[i],
                                                            errorCOh[i],
                                                            fitCOc[i],
                                                            errorCOc[i],
                                                            fitCOw[i],
                                                            errorCOw[i],
                                                            Cloud
                                                            ]     

    if n_CO == 3:
        
        CO_p0 = np.tile(CO_p0, n_CO)
        CO_bounds = np.tile(CO_bounds, n_CO)
        parametersCO, covarianceCO = curve_fit(tripleGauss, velCO, TaCO, CO_p0, bounds = CO_bounds) 
            
        fitCOh = [parametersCO[0], parametersCO[3], parametersCO[6]]
        fitCOc = [parametersCO[1], parametersCO[4], parametersCO[7]]
        fitCOw = [parametersCO[2], parametersCO[5], parametersCO[8]] 
        perrCO = np.sqrt(np.diag(covarianceCO))
        errorCOh = [perrCO[0], perrCO[3], perrCO[6]]
        errorCOc = [perrCO[1], perrCO[4], perrCO[7]]
        errorCOw = [perrCO[2], perrCO[5], perrCO[8]]
        
        
        for i in range(n_CO):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        'CO',
                                                        i+1,
                                                        fitCOh[i],
                                                        errorCOh[i],
                                                        fitCOc[i],
                                                        errorCOc[i],
                                                        fitCOw[i],
                                                        errorCOw[i],
                                                        Cloud
                                                        ]
            

    


               
    resultsTable = resultsTable.drop(0)
    return resultsTable
    #df = fit_coord(...)
    #df.save_csv()

    #files = glob.glob(*.csv)
    #for file in files:
        #df.read_csv(...)
        #df_full.concatenate(df)
        #df_full.summary()
        
    
'''
SETTING UP THE BIG TABLE:

df = fit_coord(l, b, params)
df.to_csv("l+b_fitted.csv")
df

ELSEWHERE (another cell/notebook)

files = glob.glob(fits/*csv)

df_full = pd.df()

#concate each df:
for file in files:
    df.read_csv(file)
    df_full.concatenate(file)
'''

   
        
        
#replace NaN's w/ zeros
#add in sigTCO for CO fitting
#for fwhm and center velocity for CO, use OH parameter instead of HI
        
        
        
        

'\nSETTING UP THE BIG TABLE:\n\ndf = fit_coord(l, b, params)\ndf.to_csv("l+b_fitted.csv")\ndf\n\nELSEWHERE (another cell/notebook)\n\nfiles = glob.glob(fits/*csv)\n\ndf_full = pd.df()\n\n#concate each df:\nfor file in files:\n    df.read_csv(file)\n    df_full.concatenate(file)\n'

In [7]:
#fit_coord(119070, 2800, n_HI = 2)
df = fit_coord(119070, 2800, n_HI = 2, n_67 = 1, n_65 = 1, n_CO = 1, HI_p0 = (20, 10, 15), HI_bounds = ([0, -20, 1], [30, 30, 30]), Cloud = 'Polaris')
df.to_csv("119070p2800_fitted.csv")
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),Cloud
1,119070,2800,HI,1,25.108944,0.262239,-4.629767,0.021005,1.847756,0.024123,Polaris
2,119070,2800,HI,2,6.805908,0.134199,-9.986011,0.218256,11.427903,0.200721,Polaris
3,119070,2800,67,1,0.086602,0.002528,-4.358241,0.037192,1.103239,0.037201,Polaris
4,119070,2800,67,2,0.002129,0.000000,-4.629767,0.000000,1.847756,0.000000,Polaris
5,119070,2800,65,1,0.043520,0.002556,-4.251790,0.073304,1.081164,0.073336,Polaris
6,119070,2800,65,2,0.002643,0.000000,-4.629767,0.000000,1.847756,0.000000,Polaris
7,119070,2800,CO,1,1.613065,0.125558,-3.779779,0.077823,0.865799,0.077823,Polaris
8,119070,2800,CO,2,0.136236,0.000000,-4.251790,0.000000,1.081164,0.000000,Polaris


In [8]:
df = fit_coord(119210, 2000, n_HI = 3, n_67 = 2, n_65 = 1, n_CO = 1, OH67_p0 = (1, -1, 9), HI_p0 = (1, -8, 30),
          OH67_bounds = ([0, -20, 0], [3, 4, 10]), 
            HI_bounds = ([0, -25, 0], [80, 25, 30]), Cloud = 'Polaris')
df.to_csv("119210p2000_fitted.csv")
df

#work on this fit a bit more

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),Cloud
1,119210,2000,HI,1,21.241550,0.469158,-1.532376,0.156758,4.097694,0.165828,Polaris
2,119210,2000,HI,2,25.239016,0.566368,-11.563658,0.108096,2.904294,0.098246,Polaris
3,119210,2000,HI,3,4.449115,0.216782,-23.845754,1.507364,24.665624,1.153248,Polaris
4,119210,2000,67,1,0.014840,0.001629,-2.937416,0.619626,2.897271,0.516488,Polaris
5,119210,2000,67,2,0.038055,0.004090,0.625262,0.072361,0.747977,0.098092,Polaris
6,119210,2000,67,3,0.003150,0.000000,-1.532376,0.000000,4.097694,0.000000,Polaris
7,119210,2000,65,1,0.015542,0.002237,0.695469,0.234513,1.411649,0.234513,Polaris
8,119210,2000,65,2,0.002410,0.000000,-1.532376,0.000000,4.097694,0.000000,Polaris
9,119210,2000,65,3,0.002410,0.000000,-11.563658,0.000000,2.904294,0.000000,Polaris
10,119210,2000,CO,1,0.640718,0.144064,0.650001,0.190147,0.732573,0.190346,Polaris


In [9]:
#fit_coord(120500, 1863, n_HI = 3)
df = fit_coord(120500, 1863, n_HI = 3, HI_p0 = (1, 1, 1), 
          HI_bounds = ([0, -12, 1], [30, 5, 50]), Cloud = 'Polaris')
df.to_csv("120500p1863_fitted.csv")
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),Cloud
1,120500,1863,HI,1,16.151980,1.153345,-9.055389,0.377908,9.278406,0.337238,Polaris
2,120500,1863,HI,2,29.351272,1.058692,-1.140389,0.061317,1.947711,0.081703,Polaris
3,120500,1863,HI,3,7.999107,1.201906,-9.832292,0.209921,1.888819,0.323243,Polaris
4,120500,1863,67,1,0.098978,0.002665,-0.888053,0.030852,0.991228,0.030783,Polaris
5,120500,1863,67,2,0.002534,0.000000,-9.055389,0.000000,9.278406,0.000000,Polaris
6,120500,1863,67,3,0.002534,0.000000,-1.140389,0.000000,1.947711,0.000000,Polaris
7,120500,1863,65,1,0.056665,0.002827,-0.806004,0.050877,0.882485,0.050795,Polaris
8,120500,1863,65,2,0.002528,0.000000,-9.055389,0.000000,9.278406,0.000000,Polaris
9,120500,1863,65,3,0.002528,0.000000,-1.140389,0.000000,1.947711,0.000000,Polaris
10,120500,1863,CO,1,2.971228,0.136213,-0.813725,0.036567,0.690755,0.036565,Polaris


In [10]:
df = fit_coord(120500, 2963, n_67 = 3, n_HI = 3, OH67_p0 = [1, 0, 2], CO_p0 = [1, -5, 0.5], 
           OH65_p0 = [-1, -8, 1], OH67_bounds = ([0, -10, 0], [3, 5, 5]), 
           OH65_bounds = ([-1, -25, 0], [0, 0, 5]), HI_bounds = ([0, -25, 0], [20, 25, 50]), Cloud = 'Polaris')
df.to_csv("120500p2963_fitted.csv")
df

#look into 65 fit

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),Cloud
1,120500,2963,HI,1,7.081242,0.986327,-7.097378,0.088163,1.261996,0.157982,Polaris
2,120500,2963,HI,2,18.835690,0.399500,-3.127904,0.163236,3.312519,0.123917,Polaris
3,120500,2963,HI,3,4.760325,0.173409,-12.611326,0.567041,15.075730,0.426820,Polaris
4,120500,2963,67,1,0.014573,0.001616,-3.185828,0.551547,3.041903,0.444240,Polaris
5,120500,2963,67,2,0.030204,0.003895,-6.391252,0.090201,0.627027,0.106419,Polaris
6,120500,2963,67,3,0.002072,0.000000,4.963956,0.000000,0.025058,0.000000,Polaris
7,120500,2963,65,1,-0.083266,0.005709,-8.052769,0.055417,0.176147,0.019623,Polaris
8,120500,2963,65,2,0.002807,0.000000,-7.097378,0.000000,1.261996,0.000000,Polaris
9,120500,2963,65,3,0.002807,0.000000,-3.127904,0.000000,3.312519,0.000000,Polaris
10,120500,2963,CO,1,1.475331,0.128105,-6.307529,0.057041,0.563438,0.056081,Polaris


In [11]:
df = fit_coord(121070, 2175, n_65 = 2, n_67 = 2, n_CO = 2, n_HI = 2,
           OH65_p0 = (1, 0, 2),
           CO_bounds = ([0, -10, 0], [5, 1, 5]),
          HI_bounds = ([0, -15, 0], [20, 5, 30]), 
           OH67_bounds = ([0, -10, 0], [2, 3, 6]), 
           OH65_bounds = ([0, -10, 0], [2, 3, 3]), Cloud = 'Polaris')
df.to_csv("121070p2175_fitted.csv")
df

[0.         0.         0.         0.00306497 0.03411198 0.02953876]


,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),Cloud
1,121070,2175,HI,1,9.199653,0.514994,-2.795865,0.134951,2.573746,0.172622,Polaris
2,121070,2175,HI,2,9.894485,0.414983,-6.992074,0.301406,9.146891,0.253433,Polaris
3,121070,2175,67,1,0.036667,0.001935,-3.012290,0.287764,2.051381,0.246617,Polaris
4,121070,2175,67,2,0.122155,0.005057,-6.000259,0.023289,0.666111,0.046174,Polaris
5,121070,2175,65,1,0.001942,0.000000,2.999946,0.000000,0.025500,0.000000,Polaris
6,121070,2175,65,2,0.077346,0.003065,-5.794755,0.034112,-5.794755,0.029539,Polaris
7,121070,2175,CO,1,3.861257,0.274409,-6.122722,0.043057,0.780493,0.048677,Polaris
8,121070,2175,CO,2,1.935918,0.091977,-3.416228,0.234138,1.650391,0.196384,Polaris


In [12]:
df = fit_coord(121930, 1975, n_HI = 2, n_67 = 2, OH67_p0 = (1, 0, 2), HI_p0 = (1.3, -10, 4),
           OH65_p0 = (1, 0, 2), 
           OH65_bounds = ([0, 0, 0], [1, 10, 5]), 
           OH67_bounds = ([0, -6, 0], [1, 10, 5]), 
           HI_bounds = ([0, -20, 1], [40, 5, 50]), Cloud = 'Polaris') 

df.to_csv("121930p1975_fitted.csv")
df
#high error????

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),Cloud
1,121930,1975,HI,1,19.510653,1.101174,-0.612584,0.303870,3.564322,0.254745,Polaris
2,121930,1975,HI,2,18.323957,0.657323,-12.521182,0.460614,5.477668,0.425020,Polaris
3,121930,1975,67,1,0.118336,1433.150078,1.323986,11.774807,0.249405,876.700096,Polaris
4,121930,1975,67,2,0.015215,0.003054,-2.997574,0.174187,0.717521,0.162056,Polaris
5,121930,1975,65,1,0.057793,732.974889,1.298861,66.455797,0.255006,979.363894,Polaris
6,121930,1975,65,2,0.002524,0.000000,-0.612584,0.000000,3.564322,0.000000,Polaris
7,121930,1975,CO,1,0.859033,0.160622,1.332860,0.119604,0.569361,0.125836,Polaris
8,121930,1975,CO,2,0.160260,0.000000,1.298861,0.000000,0.255006,0.000000,Polaris


In [13]:
df = fit_coord(122220, 1875, n_HI = 3, HI_p0 = (1, -10, 25), CO_p0 = (1, 2, 0.5),
          CO_bounds = ([0, 0, 0.1], [3, 3, 3]), 
          HI_bounds = ([0, -20, 1], [30, 20, 50]), Cloud = 'Polaris')
df.to_csv("122220p1875_fitted.csv")
df
#CHECK LATER ON CO ERROR (basically just noise as seen in plot)

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),Cloud
1,122220,1875,HI,1,10.443772,1.629264,-12.628355,0.225800,1.316190,0.258288,Polaris
2,122220,1875,HI,2,19.828186,1.001218,-8.541770,0.541676,9.482401,0.378219,Polaris
3,122220,1875,HI,3,5.199265,1.425286,0.066284,0.683497,2.496087,0.858417,Polaris
4,122220,1875,67,1,0.027175,0.002953,1.999606,0.101372,0.803392,0.100404,Polaris
5,122220,1875,67,2,0.002125,0.000000,-12.628355,0.000000,1.316190,0.000000,Polaris
6,122220,1875,67,3,0.002125,0.000000,-8.541770,0.000000,9.482401,0.000000,Polaris
7,122220,1875,65,1,0.013580,0.002473,1.981341,0.242752,1.154046,0.242751,Polaris
8,122220,1875,65,2,0.002410,0.000000,-12.628355,0.000000,1.316190,0.000000,Polaris
9,122220,1875,65,3,0.002410,0.000000,-8.541770,0.000000,9.482401,0.000000,Polaris
10,122220,1875,CO,1,2.180064,4531.473308,1.964676,19.727884,0.192040,139.530787,Polaris


In [14]:
df = fit_coord(122500, 2900, n_HI = 3, n_67 = 3, 
           HI_p0 = (1, -10, 25), OH67_p0 = (1, -3, 4),
           CO_p0 = (2, -4, 3),
           OH67_bounds = ([0, -8, 1], [5, 3, 10]), 
           CO_bounds = ([0, -10, 1], [5, -1, 5]), 
          HI_bounds = ([0, -30, 1], [20, -3, 30]), Cloud = 'Polaris')
df.to_csv = ("122500p2900_fitted.csv")
df
#issue with OH bounds and CO 

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),Cloud
1,122500,2900,HI,1,7.235595,0.474119,-23.078703,0.192101,2.701801,0.231203,Polaris
2,122500,2900,HI,2,4.282997,0.382755,-13.187170,0.994570,15.314910,0.844555,Polaris
3,122500,2900,HI,3,14.775205,0.467533,-5.189047,0.100601,3.018431,0.124775,Polaris
4,122500,2900,67,1,0.006047,0.002686,0.409367,0.522681,1.000000,0.547616,Polaris
5,122500,2900,67,2,0.011433,0.002728,-3.772146,0.328044,1.000000,0.372935,Polaris
6,122500,2900,67,3,0.015803,0.002697,-7.186639,0.238710,1.000000,0.253389,Polaris
7,122500,2900,65,1,0.012392,0.002303,-6.669039,0.285713,1.331156,0.285713,Polaris
8,122500,2900,65,2,0.002505,0.000000,-23.078703,0.000000,2.701801,0.000000,Polaris
9,122500,2900,65,3,0.002505,0.000000,-13.187170,0.000000,15.314910,0.000000,Polaris
10,122500,2900,CO,1,0.314455,0.075126,-5.528401,0.451912,1.638152,0.451912,Polaris


In [15]:
df = fit_coord(123360, 2075, n_HI = 3, HI_p0 = (2, -3, 1), n_65 = 2, OH65_p0 = (1, 0, 2), 
           n_67 = 3, OH67_p0 = (1, 0, 4), n_CO = 2, CO_p0 = (1, 0, 3)
          , OH67_bounds = ([0, -10, 0], [4, 5, 10]), OH65_bounds = ([0, -10, 0], [4, 5, 10]),
          CO_bounds = ([0, -5, 0], [4, 4, 5]), HI_bounds = ([0, -20, 0], [30, 20, 10]), Cloud = 'Polaris')
df.to_csv = ("123360p2075_fitted.csv")
df

#67 temperature peak questionable

[1.71812551e-03 5.45915377e-01 4.12649958e-01 2.57715697e+02
 6.14131004e+01 2.36079067e+02]


,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),Cloud
1,123360,2075,HI,1,25.697443,1.364047,-3.592407,0.114687,1.981046,0.135200,Polaris
2,123360,2075,HI,2,19.038661,0.766518,4.564557,0.175270,3.071973,0.192284,Polaris
3,123360,2075,HI,3,18.089185,0.598028,-12.401738,0.341516,5.083314,0.339971,Polaris
4,123360,2075,67,1,0.028700,0.002614,-0.495989,0.173152,1.359862,0.278183,Polaris
5,123360,2075,67,2,0.033117,1169.275659,-3.382521,181.065455,0.252704,2659.506661,Polaris
6,123360,2075,67,3,0.066777,0.003256,2.828732,0.059083,0.775618,0.054645,Polaris
7,123360,2075,65,1,0.013952,0.001718,-0.045986,0.545915,3.121495,0.412650,Polaris
8,123360,2075,65,2,0.093760,257.715697,3.042372,61.413100,3.042372,236.079067,Polaris
9,123360,2075,65,3,0.002343,0.000000,-3.592407,0.000000,1.981046,0.000000,Polaris
10,123360,2075,CO,1,1.623683,0.091746,-0.481580,0.086190,1.122112,0.094523,Polaris


In [16]:
df = fit_coord(123500, 1963, n_67 = 2, n_HI = 3, n_CO = 2, HI_p0 = (15, 0, 5), OH65_p0 = (2, 0, 2),
           CO_p0 = (.4, -2, .4), CO_bounds = ([0, -10, 0], [2, 0, 10]), OH67_p0 = (1, 0, 2), 
          OH65_bounds = ([0, -10, 0], [5, 1, 5]), 
           HI_bounds= ([10, -30, 1], [80, 25, 15]), OH67_bounds = ([0, -10, 0], [5, 1, 5]), Cloud = 'Polaris')

df.to_csv("123500p1963_fitted.csv")
df
#cant get 65 to fit

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),Cloud
1,123500,1963,HI,1,18.089110,0.598024,-12.401713,0.341525,5.083383,0.339978,Polaris
2,123500,1963,HI,2,25.697267,1.364069,-3.592402,0.114687,1.981038,0.135201,Polaris
3,123500,1963,HI,3,19.038646,0.766519,4.564559,0.175269,3.071973,0.192284,Polaris
4,123500,1963,67,1,0.004447,0.001428,1.000000,2.744662,5.000000,1.931647,Polaris
5,123500,1963,67,2,0.032438,0.003053,-4.054945,0.116178,1.327152,0.152601,Polaris
6,123500,1963,67,3,0.002180,0.000000,-12.401713,0.000000,5.083383,0.000000,Polaris
7,123500,1963,65,1,0.017095,101650.853882,0.303705,37148.906546,0.220367,296484.182542,Polaris
8,123500,1963,65,2,0.001962,0.000000,-12.401713,0.000000,5.083383,0.000000,Polaris
9,123500,1963,65,3,0.001962,0.000000,-3.592402,0.000000,1.981038,0.000000,Polaris
10,123500,1963,CO,1,0.405409,0.219062,-5.356204,0.223274,0.466562,0.298617,Polaris


In [17]:
df = fit_coord(125070, 1950, n_HI = 2, HI_p0 = (10, 1, 20), 
               HI_bounds = ([0, -30, 0], [20, 10, 30]), Cloud = 'Polaris')
df.to_csv("125070p1950_fitted.csv")
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),Cloud
1,125070,1950,HI,1,8.403221,0.719006,4.079471,0.195456,3.464665,0.308162,Polaris
2,125070,1950,HI,2,16.459169,0.269667,-8.096963,0.399496,10.082047,0.284664,Polaris
3,125070,1950,67,1,0.106404,0.003160,0.660790,0.024930,0.620396,0.020450,Polaris
4,125070,1950,67,2,0.002858,0.000000,4.079471,0.000000,3.464665,0.000000,Polaris
5,125070,1950,65,1,0.059559,0.003096,0.686141,0.047755,0.555600,0.032003,Polaris
6,125070,1950,65,2,0.001947,0.000000,4.079471,0.000000,3.464665,0.000000,Polaris
7,125070,1950,CO,1,1.962675,0.147545,0.680557,0.051534,0.601743,0.052906,Polaris
8,125070,1950,CO,2,0.132241,0.000000,0.686141,0.000000,0.555600,0.000000,Polaris


In [18]:
df = fit_coord(125220, 3250, OH67_bounds = ([0, -5, 0], [3, 5, 5]), Cloud = 'Polaris')
df.to_csv("125220p3250_fitted.csv")
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),Cloud
1,125220,3250,HI,1,36.662619,0.776355,1.152745,0.067798,2.775750,0.067794,Polaris
2,125220,3250,67,1,0.120393,0.002593,0.069041,0.026084,1.048728,0.026070,Polaris
3,125220,3250,65,1,0.068332,0.002713,0.234129,0.044017,0.961648,0.044164,Polaris
4,125220,3250,CO,1,0.863866,0.103696,0.389683,0.137698,0.993339,0.137698,Polaris


In [19]:
df = fit_coord(125500, 2200, n_HI = 2, n_67 = 2, n_65 = 2, OH67_p0 = (1, 0, 2)
          , OH67_bounds = ([0, -5, 0], [1.5, 5, 5]), HI_bounds = ([0, -20, 0], [20, 15, 30]),
          OH65_bounds = ([0, -5, 0], [1.5, 5, 5]), Cloud = 'Polaris') 
df.to_csv("125500p2200_fitted.csv")
df

#note, double check this is in polaris later!!

[0.00318181 0.75149387 0.53574244 0.00453271 0.10368592 0.12489149]


,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),Cloud
1,125500,2200,HI,1,14.435097,0.377697,-2.672325,0.092363,3.392045,0.111443,Polaris
2,125500,2200,HI,2,6.079058,0.242600,-9.139501,0.554362,16.841603,0.536627,Polaris
3,125500,2200,67,1,0.016239,0.003111,-3.185533,0.476811,2.474600,0.334488,Polaris
4,125500,2200,67,2,0.056400,0.004332,-4.709768,0.055014,0.739636,0.060019,Polaris
5,125500,2200,65,1,0.009469,0.003182,-3.344392,0.751494,2.205562,0.535742,Polaris
6,125500,2200,65,2,0.029500,0.004533,-4.582643,0.103686,-4.582643,0.124891,Polaris
7,125500,2200,CO,1,2.437001,0.095121,-4.804518,0.038853,0.911051,0.043082,Polaris
8,125500,2200,CO,2,0.073953,0.000000,-3.344392,0.000000,2.205562,0.000000,Polaris


In [20]:
df = fit_coord(127500, 2088, n_67 = 2, n_65 = 2, 
           n_HI = 3, OH67_p0 = (4, 0, 3), 
           OH65_p0 = (2, 0, 3), 
           HI_p0 = (10, 0, 30), 
           OH65_bounds = ([0, -15, 1], [2, 10, 10]), 
           OH67_bounds = ([0, -15, 1], [15, 5, 10]),
          HI_bounds = ([0, -25, 1], [30, 18, 40]), Cloud = 'Polaris')
df.to_csv("127500p2088_fitted.csv")
df

[0.00180765 0.18123807 0.18665192 0.00157968 1.03741926 1.08720535]


,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),Cloud
1,127500,2088,HI,1,24.285441,0.601601,-2.632001,0.066571,2.669135,0.072193,Polaris
2,127500,2088,HI,2,16.973269,0.247301,6.196843,0.135649,4.078036,0.123679,Polaris
3,127500,2088,HI,3,11.197022,0.173359,-13.794739,0.298294,7.508597,0.269080,Polaris
4,127500,2088,67,1,0.006635,0.001598,3.496422,1.835215,6.693859,1.283232,Polaris
5,127500,2088,67,2,0.049094,0.002518,-1.098451,0.090182,1.737854,0.110832,Polaris
6,127500,2088,67,3,0.002654,0.000000,-2.632001,0.000000,2.669135,0.000000,Polaris
7,127500,2088,65,1,0.025791,0.001808,-1.390455,0.181238,2.183250,0.186652,Polaris
8,127500,2088,65,2,0.005158,0.001580,9.148195,1.037419,9.148195,1.087205,Polaris
9,127500,2088,65,3,0.002219,0.000000,-2.632001,0.000000,2.669135,0.000000,Polaris
10,127500,2088,CO,1,2.598015,0.103059,-0.610891,0.059003,1.288129,0.059003,Polaris


In [21]:
df = fit_coord(137160, 4300, OH67_p0 = (1, 10, 1), n_HI = 2,
          HI_bounds = ([0, 0, 1], [20 ,15, 15]), Cloud = 'Ursa Major')
df.to_csv("137160p4300_fitted.csv")
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),Cloud
1,137160,4300,HI,1,2.592075e+00,1.587096e-01,2.875547e-16,8.080451e-01,10.876046,6.581775e-01,Ursa Major
2,137160,4300,HI,2,1.121861e+01,3.570247e-01,7.849033e+00,6.140464e-02,1.756083,6.965312e-02,Ursa Major
3,137160,4300,67,1,1.004865e-02,2.925679e-03,8.702887e+00,2.770823e-01,0.823589,2.767421e-01,Ursa Major
4,137160,4300,67,2,1.740240e-03,0.000000e+00,2.875547e-16,0.000000e+00,10.876046,0.000000e+00,Ursa Major
5,137160,4300,65,1,3.101224e-03,2.081433e-03,-1.008849e+00,1.263051e+00,1.629755,1.263051e+00,Ursa Major
6,137160,4300,65,2,2.298990e-03,0.000000e+00,2.875547e-16,0.000000e+00,10.876046,0.000000e+00,Ursa Major
7,137160,4300,CO,1,-1.617513e-19,2.212943e-20,1.130883e-09,1.319636e-10,1.000000,8.437336e-10,Ursa Major
8,137160,4300,CO,2,0.000000e+00,0.000000e+00,-1.008849e+00,0.000000e+00,1.629755,0.000000e+00,Ursa Major


In [22]:
df = fit_coord(142210, 2325, n_HI = 3, HI_p0 = (1, 0, 8), 
           HI_bounds = ([0, -30, 0], [20, 5, 50]), Cloud = 'Camelopardalis')
df.to_csv("142210p2325_fitted.csv")
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),Cloud
1,142210,2325,HI,1,0.818782,0.095787,-28.850766,5.007798,36.470092,3.877945,Camelopardalis
2,142210,2325,HI,2,14.290681,0.283359,-3.126089,0.110026,8.025420,0.147917,Camelopardalis
3,142210,2325,HI,3,19.395988,0.384563,-1.330761,0.033308,1.744758,0.042756,Camelopardalis
4,142210,2325,67,1,0.084863,0.002897,-2.042813,0.032881,0.820334,0.031870,Camelopardalis
5,142210,2325,67,2,0.002214,0.000000,-28.850766,0.000000,36.470092,0.000000,Camelopardalis
6,142210,2325,67,3,0.002214,0.000000,-3.126089,0.000000,8.025420,0.000000,Camelopardalis
7,142210,2325,65,1,0.043649,0.002917,-1.955868,0.063117,0.789529,0.059132,Camelopardalis
8,142210,2325,65,2,0.002606,0.000000,-28.850766,0.000000,36.470092,0.000000,Camelopardalis
9,142210,2325,65,3,0.002606,0.000000,-3.126089,0.000000,8.025420,0.000000,Camelopardalis
10,142210,2325,CO,1,2.504879,0.131268,-2.169310,0.043053,0.711052,0.043011,Camelopardalis


In [23]:
df = fit_coord(143000, 3850, n_HI = 2, HI_bounds = ([0, -10, 0], [20, 15, 30]), Cloud = 'Ursa Major')
df.to_csv("143000p3850_fitted.csv")
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),Cloud
1,143000,3850,HI,1,12.805758,1.821154,2.037586,0.150874,1.359361,0.202025,Ursa Major
2,143000,3850,HI,2,17.597695,1.611608,4.112797,0.276431,3.666836,0.175858,Ursa Major
3,143000,3850,67,1,0.097741,0.002780,2.225671,0.030040,0.918526,0.030299,Ursa Major
4,143000,3850,67,2,0.002283,0.000000,2.037586,0.000000,1.359361,0.000000,Ursa Major
5,143000,3850,65,1,0.053961,0.002779,2.292664,0.054513,0.921137,0.055060,Ursa Major
6,143000,3850,65,2,0.002123,0.000000,2.037586,0.000000,1.359361,0.000000,Ursa Major
7,143000,3850,CO,1,2.491926,0.101556,2.302071,0.037419,0.795076,0.037414,Ursa Major
8,143000,3850,CO,2,0.118068,0.000000,2.292664,0.000000,0.921137,0.000000,Ursa Major


In [24]:
df = fit_coord(146070, 1775, n_HI = 2, OH67_bounds = ([0, -5, 0], [20, 5, 5])
          , OH65_bounds = ([0, -5, 0], [20, 5, 5]), HI_bounds = ([0, -20, 0], [20, 8, 30]), Cloud = 'Camelopardalis')
df.to_csv("146070p1775_fitted.csv")
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),Cloud
1,146070,1775,HI,1,17.389566,0.882948,-1.931734,0.210942,5.902981,0.168969,Camelopardalis
2,146070,1775,HI,2,20.000000,0.976255,0.743239,0.078732,1.976837,0.105158,Camelopardalis
3,146070,1775,67,1,0.230267,0.003001,0.487612,0.011596,0.723211,0.010439,Camelopardalis
4,146070,1775,67,2,0.001917,0.000000,-1.931734,0.000000,5.902981,0.000000,Camelopardalis
5,146070,1775,65,1,0.169170,0.002986,0.591286,0.015728,0.687076,0.013057,Camelopardalis
6,146070,1775,65,2,0.002290,0.000000,-1.931734,0.000000,5.902981,0.000000,Camelopardalis
7,146070,1775,CO,1,4.350812,0.114572,0.258126,0.027344,0.899220,0.027344,Camelopardalis
8,146070,1775,CO,2,0.139482,0.000000,0.591286,0.000000,0.687076,0.000000,Camelopardalis


In [25]:
df = fit_coord(146200, 3963, n_HI = 2, HI_p0 = (1, 0, 5), HI_bounds = ([0, -10, 0], [20, 15, 30]), Cloud = 'Ursa Major')
df.to_csv("146200p3963_fitted.csv")
df

#weird HI bounds behavior when varying upper bounds

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),Cloud
1,146200,3963,HI,1,9.588168,0.809131,-2.501404,0.378141,3.087574,0.413438,Ursa Major
2,146200,3963,HI,2,15.570900,1.043702,5.265058,0.178000,1.844490,0.173670,Ursa Major
3,146200,3963,67,1,0.022717,0.002383,-0.098259,0.150440,1.241909,0.150438,Ursa Major
4,146200,3963,67,2,0.002467,0.000000,-2.501404,0.000000,3.087574,0.000000,Ursa Major
5,146200,3963,65,1,0.017295,0.003539,0.399105,0.142963,0.695627,0.177370,Ursa Major
6,146200,3963,65,2,0.002187,0.000000,-2.501404,0.000000,3.087574,0.000000,Ursa Major
7,146200,3963,CO,1,1.779890,0.092886,0.001515,0.049196,0.816434,0.049200,Ursa Major
8,146200,3963,CO,2,0.111297,0.000000,0.399105,0.000000,0.695627,0.000000,Ursa Major


In [26]:
df = fit_coord(146930, 2075, Cloud = 'Camelopardalis')
df.to_csv("146930p2075_fitted.csv")
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),Cloud
1,146930,2075,HI,1,37.110928,0.772866,-1.210916,0.072475,3.014247,0.072489,Camelopardalis
2,146930,2075,67,1,0.057882,0.002881,-0.134523,0.048703,0.841744,0.048108,Camelopardalis
3,146930,2075,65,1,0.033501,0.004148,0.136408,0.068494,0.642267,0.102316,Camelopardalis
4,146930,2075,CO,1,0.697665,0.092378,0.055885,0.106260,0.695648,0.106522,Camelopardalis


In [27]:
df = fit_coord(147000, 4038, n_CO = 2, n_67 = 2, n_HI = 2, n_65 = 2, OH67_bounds = ([0, -15, 0], [1, 3, 5])
          , OH65_bounds = ([0, -5, 0], [1, 3, 5]), CO_bounds = ([0, -8, 0], [2, 5, 5]), HI_bounds = ([0, -50, 0], [20, 15, 50]), Cloud = 'Ursa Major')
df.to_csv("147000p4038_fitted.csv")
df


[0.00410005 0.16815495 0.21068089 0.00273594 0.64738753 0.80559587]


,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),Cloud
1,147000,4038,HI,1,11.054594,0.177355,0.713808,0.081339,4.668752,0.093314,Ursa Major
2,147000,4038,HI,2,0.875852,0.064871,-34.595443,3.398962,32.054796,3.135393,Ursa Major
3,147000,4038,67,1,0.007669,0.002783,1.023215,0.572774,0.931544,0.568081,Ursa Major
4,147000,4038,67,2,0.041632,0.002765,-1.883542,0.105972,0.942037,0.105079,Ursa Major
5,147000,4038,65,1,0.016683,0.004100,-1.267397,0.168155,0.593552,0.210681,Ursa Major
6,147000,4038,65,2,0.008834,0.002736,-0.919058,0.647388,-0.919058,0.805596,Ursa Major
7,147000,4038,CO,1,1.542826,0.100167,-1.863680,0.063116,0.841038,0.064971,Ursa Major
8,147000,4038,CO,2,0.607147,0.128383,1.116347,0.126575,0.495881,0.123451,Ursa Major


In [28]:
df = fit_coord(147200, 4075, n_67 = 2, n_65 = 1, n_CO = 2, n_HI = 2,
           OH65_bounds = ([0, -15, 0], [2, 8, 5]),
           OH67_bounds = ([0, -15, 0], [2, 8, 5]),
          CO_bounds = ([0, 0, 0], [2, 10, 2]), HI_bounds = ([0, -15, 0], [50, 30, 50]), Cloud = 'Ursa Major')  
df.to_csv("147200p4075_fitted.csv")
df

#issue when I apply bounds to 65

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),Cloud
1,147200,4075,HI,1,6.923547,0.622045,3.855903,0.095112,1.031313,0.103485,Ursa Major
2,147200,4075,HI,2,17.243276,0.393702,1.226613,0.100445,3.903026,0.071469,Ursa Major
3,147200,4075,67,1,0.076728,0.002969,3.683673,0.034997,0.731117,0.031112,Ursa Major
4,147200,4075,67,2,0.025083,0.002510,-1.748470,0.129352,1.119450,0.129375,Ursa Major
5,147200,4075,65,1,0.054946,0.003199,3.812877,0.047296,0.683667,0.045846,Ursa Major
6,147200,4075,65,2,0.001940,0.000000,3.855903,0.000000,1.031313,0.000000,Ursa Major
7,147200,4075,CO,1,1.999999,54.077501,3.789920,1.946731,0.689066,1.123626,Ursa Major
8,147200,4075,CO,2,1.999999,54.099328,3.668363,1.484570,0.603640,1.225258,Ursa Major


In [29]:
df = fit_coord(148200, 3863, n_HI = 2, HI_p0 = (1, 0, 5), 
           HI_bounds = ([0, -10, 1], [20, 10, 20]), Cloud = 'Ursa Major')
df.to_csv("148200p3863_fitted.csv")
df


,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),Cloud
1,148200,3863,HI,1,8.934638,1.033549,1.100517,0.460988,5.912545,0.459897,Ursa Major
2,148200,3863,HI,2,7.812903,1.254322,-0.933010,0.231440,1.572206,0.301958,Ursa Major
3,148200,3863,67,1,0.047720,0.002417,-1.827805,0.070603,1.207130,0.070602,Ursa Major
4,148200,3863,67,2,0.002580,0.000000,1.100517,0.000000,5.912545,0.000000,Ursa Major
5,148200,3863,65,1,0.041634,0.002856,-2.200168,0.068436,0.860289,0.067855,Ursa Major
6,148200,3863,65,2,0.002533,0.000000,1.100517,0.000000,5.912545,0.000000,Ursa Major
7,148200,3863,CO,1,1.576454,0.102998,-2.316782,0.052379,0.693315,0.052232,Ursa Major
8,148200,3863,CO,2,0.113045,0.000000,-2.200168,0.000000,0.860289,0.000000,Ursa Major


In [30]:
df = fit_coord(148210, 2350, n_HI = 2, HI_p0 = (1, 0, 5), HI_bounds = ([0, -20, 0], [20, 15, 30]),
           OH67_p0 = (1, 1, 1),
          OH67_bounds = ([0, 0, 0], [1, 5, 2]), Cloud = 'Camelopardalis')
df.to_csv("148210p2350_fitted.csv")
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),Cloud
1,148210,2350,HI,1,11.950775,0.233022,-1.813379,0.157706,8.475079,0.148828,Camelopardalis
2,148210,2350,HI,2,10.816934,0.469056,1.905390,0.061185,1.293286,0.070372,Camelopardalis
3,148210,2350,67,1,0.066329,0.003094,1.370475,0.039619,0.671339,0.034762,Camelopardalis
4,148210,2350,67,2,0.002416,0.000000,-1.813379,0.000000,8.475079,0.000000,Camelopardalis
5,148210,2350,65,1,0.044365,0.003422,1.283639,0.059285,0.609142,0.054886,Camelopardalis
6,148210,2350,65,2,0.002576,0.000000,-1.813379,0.000000,8.475079,0.000000,Camelopardalis
7,148210,2350,CO,1,3.165647,0.141303,1.304324,0.026811,0.545670,0.029165,Camelopardalis
8,148210,2350,CO,2,0.128785,0.000000,1.283639,0.000000,0.609142,0.000000,Camelopardalis


In [31]:
df = fit_coord(150120, 4067, Cloud = 'Ursa Major')
df.to_csv("150120p4067_fitted.csv")
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),Cloud
1,150120,4067,HI,1,1.060897e+01,2.619626e-01,1.221581e+00,1.480027e-01,5.192746,1.480601e-01,Ursa Major
2,150120,4067,67,1,3.396686e-03,1.488132e-03,-3.658921e+00,1.611335e+00,3.183563,1.611335e+00,Ursa Major
3,150120,4067,65,1,2.465818e-03,1.126352e-03,1.338215e+00,2.935009e+00,5.567354,2.935009e+00,Ursa Major
4,150120,4067,CO,1,-1.617513e-19,2.212943e-20,1.130883e-09,1.319636e-10,1.000000,8.437336e-10,Ursa Major


In [32]:
df = fit_coord(151500, 1925, n_HI = 2, HI_bounds = ([0, -15, 0], [20, 15, 30]), Cloud = 'Camelopardalis')
df.to_csv("151500p1925_fitted.csv")
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),Cloud
1,151500,1925,HI,1,20.000000,1.010966,-0.746892,0.089164,1.846432,0.115768,Camelopardalis
2,151500,1925,HI,2,13.944777,0.757857,-1.122311,0.279187,8.748602,0.360982,Camelopardalis
3,151500,1925,67,1,0.022660,0.002330,-1.235308,0.154193,1.298628,0.154193,Camelopardalis
4,151500,1925,67,2,0.002332,0.000000,-0.746892,0.000000,1.846432,0.000000,Camelopardalis
5,151500,1925,65,1,0.011849,0.002902,-1.449924,0.234809,0.812572,0.225325,Camelopardalis
6,151500,1925,65,2,0.003053,0.000000,-0.746892,0.000000,1.846432,0.000000,Camelopardalis
7,151500,1925,CO,1,1.513998,0.124205,-1.399332,0.066468,0.702123,0.066562,Camelopardalis
8,151500,1925,CO,2,0.136645,0.000000,-1.449924,0.000000,0.812572,0.000000,Camelopardalis


In [33]:
df = fit_coord(156540, 3513, n_67 = 2, n_HI = 2, n_65 = 2, OH67_p0 = (1, 0, 3), OH65_p0 = (1, 0, 3)
          ,OH67_bounds = ([0, -10, 0], [1, 2, 5]), OH65_bounds = ([0, -10, 0], [5, 2, 5]),
          CO_bounds = ([0, -10, 0], [2, 0, 5]), HI_bounds = ([0, -20, 0], [30, 10, 50]), Cloud = 'Ursa Major')
df.to_csv("156540p3513_fitted.csv")
df
#weird CO behavior when bounds applied

[0.00306696 0.1533314  0.13833495 0.00231695 0.71006729 0.78543442]


,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),Cloud
1,156540,3513,HI,1,1.115452,0.106970,-20.000000,3.724495,35.333655,3.557668,Ursa Major
2,156540,3513,HI,2,14.696309,0.315815,-0.736531,0.079078,3.267921,0.084907,Ursa Major
3,156540,3513,67,1,0.017449,0.003448,0.426706,0.145291,0.666743,0.161629,Ursa Major
4,156540,3513,67,2,0.032019,0.002822,-3.067127,0.090899,0.883918,0.091281,Ursa Major
5,156540,3513,65,1,0.022313,0.003067,-3.047263,0.153331,0.781650,0.138335,Ursa Major
6,156540,3513,65,2,0.006455,0.002317,0.436483,0.710067,0.436483,0.785434,Ursa Major
7,156540,3513,CO,1,1.725987,0.114585,-3.363716,0.052656,0.687417,0.052742,Ursa Major
8,156540,3513,CO,2,0.128935,0.000000,-3.047263,0.000000,0.781650,0.000000,Ursa Major


In [34]:
df = fit_coord(156640, 3250, Cloud = 'Ursa Major')
df.to_csv("156640p3250_fitted.csv")
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s),Cloud
1,156640,3250,HI,1,23.456820,0.679447,-0.742946,0.095651,2.861469,0.095685,Ursa Major
2,156640,3250,67,1,0.001190,0.001864,-0.204542,3.671439,-2.028824,3.671440,Ursa Major
3,156640,3250,65,1,0.002425,0.001370,0.949289,2.455072,3.763955,2.455072,Ursa Major
4,156640,3250,CO,1,1.742431,0.101835,-0.815809,0.068763,1.018795,0.068763,Ursa Major
